## 대학원 합격 예측

In [9]:
import pandas as pd

data = pd.read_csv("../Data/gpascore.csv")

data.head() # 기본값: 처음 5행 출력

# 특정 행 수 지정
# data.head(n=3)  # 처음 3행
# data.head(10)   # 처음 10행

# 반대 함수
# data.tail() # 마지막 5행
# data.tail(n=3)  # 마지막 3행
# data.tail(10)   # 마지막 10행

,admit,gre,gpa,rank
0,0,380.0,3.21,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.33,4


admit: 합격여부  
gre: 영어성적(미국 및 영어권 국가들의 대학원 및 경영대학원에 입학하려는 학생들을 평가하는 시험)  
gpa: 학점  
rank: 대학원 수준  

### Preprocessing(전처리)

In [10]:
# 결측치 처리(missing value)
data.isnull().sum()

# isnull() 함수는 결측치가 있는 경우 True, 없는 경우 False를 반환
# sum() 함수는 True의 개수를 반환

admit    0
gre      1
gpa      0
rank     0
dtype: int64

In [11]:
# 결측치 데이터 삭제
data.dropna(inplace=True)

# dropna() 함수는 결측치가 있는 행을 삭제
# inplace=True 옵션은 원본 데이터를 수정하는 옵션

결측치 처리시 다른 값으로 대체  
data.fillna(100)

In [12]:
# 확인
data.isnull().sum()

admit    0
gre      0
gpa      0
rank     0
dtype: int64

In [13]:
# rank의 종류 파악
sorted(data['rank'].unique())

# unique() 함수는 중복되지 않는 값을 반환

[np.int64(1), np.int64(2), np.int64(3), np.int64(4)]

In [15]:
# 영어 점수(gre)의 최소값과 최대값
print(f'gre 최소값: {data["gre"].min()}, gre 최대값: {data["gre"].max()}')

# 학점(gpa)의 최소값과 최대값
print(f'gpa 최소값: {data["gpa"].min()}, gpa 최대값: {data["gpa"].max()}')

gre 최소값: 220.0, gre 최대값: 800.0
gpa 최소값: 2.12, gpa 최대값: 4.0


### Target Column 확인  
합격여부 데이터 빈도수

In [16]:
# 합격 여부 데이터 빈도수 확인(합격 갯수, 불합격 갯수)
(data['admit'] == 1).sum() / (data['admit'] == 0).sum()


np.float64(0.8973214285714286)

> 합격 불합격 비율이 약 90% 이므로 Target Data로 충분하다.

### Train Data 만들기

In [17]:
x = data[['gre', 'gpa', 'rank']]
x.head()

,gre,gpa,rank
0,380.0,3.21,3
1,660.0,3.67,3
2,800.0,4.00,1
3,640.0,3.19,4
4,520.0,2.33,4


In [19]:
y = data['admit']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: admit, dtype: int64

In [20]:
# 확인
print(x.count())
print(y.count())

gre     425
gpa     425
rank    425
dtype: int64
425


### Deep Learning Model 만들기

In [22]:
from tensorflow import keras
from tensorflow.keras.layers import Input # Input 레이어는 입력 데이터의 형태를 정의하는 레이어


In [27]:
# 모델 생성
model = keras.Sequential()

# 입력층
model.add(Input(shape=(3,))) # 3개의 특성(gre, gpa, rank) 입력 받음

# 은닉층
model.add(
  keras.layers.Dense(
    256, # 노드(뉴런) 개수
    activation='tanh' # 활성화 함수
  )
)

# 출력층
model.add(
  keras.layers.Dense(
    1,
    activation='sigmoid'
  )
)

# [활성화 함수]
# sigmoid: 0과 1 사이의 값을 출력  
# tanh: -1과 1 사이의 값을 출력  
# relu: 0과 양의 값을 출력  
# softmax: 여러 개의 출력 중 하나를 선택  
# LeakyReLU: 음수 영역에서 약간의 기울기를 가지는 ReLU 함수

입력 → [뉴런/노드] → 출력

    [입력층]    [은닉층]     [출력층]
      ⭕        ⭕ ⭕        ⭕
      ⭕   →   ⭕ ⭕   →    ⭕
      ⭕        ⭕ ⭕
   (3개 노드)  (6개 노드)  (2개 노드)

In [29]:
# 설정
model.compile(
  optimizer='adam', # 최적화 알고리즘
  loss='binary_crossentropy', # 손실 함수
  metrics=['accuracy'] # 평가 지표
)

# optimizer = 공부하는 방법(Learning rate을 알맞게 조정해줌)
# loss = 틀린 문제 체크
# metrics = 시험 점수

# optimizer 종류
# adam, rmsprop, adagrad, adadelta, adamax, nadam
# adam이 가장 널리 사용됨

# loss 종류
# binary_crossentropy : 이진 분류 문제에 사용(예/아니오)
# categorical_crossentropy : 다중 분류 문제에 사용(A/B/C/D..)
# mean_squared_error : 회귀 문제에 사용

# metrics 종류
# accuracy : 정확도
# precision : 정밀도
# recall : 재현률
# f1-score : 정밀도와 재현률의 조화평균


In [30]:
import numpy as np

In [34]:
# 학습 시키기

model.fit(
  np.array(x), # 입력 데이터
  np.array(y), # 출력 데이터
  epochs=1000 # 학습 횟수
)


Epoch 1/1000
 1/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6562 - loss: 0.7649

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7412 - loss: 0.5460 
Epoch 2/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7585 - loss: 0.5391
Epoch 3/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7215 - loss: 0.5642
Epoch 4/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - accuracy: 0.7184 - loss: 0.5489
Epoch 5/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7569 - loss: 0.5199  
Epoch 6/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7794 - loss: 0.5095
Epoch 7/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7560 - loss: 0.5281
Epoch 8/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - accuracy: 0.6986 - loss: 0.5628
Epoch 9/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6751 - loss: 0.5979
Epoch 10/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7793 - loss: 0.5012  
Epoch 11/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - accuracy: 0.7727 - loss: 0.5121
Epoch 12/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step -

In [35]:
# 평가하기
model.evaluate(
  np.array(x),
  np.array(y)
)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7169 - loss: 0.5363


[0.5458006858825684, 0.7082353234291077]

---
### 예측하기

In [40]:
model.predict(
  np.array([
    [300, 2.7, 1], # 학생1: GRE 300, GPA 2.7, 대학원 수준 1   
    [300, 2.7, 2], # 학생2: GRE 300, GPA 2.7, 대학원 수준 2
    [300, 2.7, 3], # 학생3: GRE 300, GPA 2.7, 대학원 수준 3
    [300, 2.7, 4], # 학생4: GRE 300, GPA 2.7, 대학원 수준 4
    [500, 3.0, 1], # 학생5: GRE 500, GPA 3.0, 대학원 수준 1
    [500, 3.0, 2], # 학생6: GRE 500, GPA 3.0, 대학원 수준 2
    [500, 3.0, 3], # 학생7: GRE 500, GPA 3.0, 대학원 수준 3
    [500, 3.0, 4], # 학생8: GRE 500, GPA 3.0, 대학원 수준 4 
    [700, 3.5, 1], # 학생9: GRE 700, GPA 3.5, 대학원 수준 1
    [700, 3.5, 2], # 학생10: GRE 700, GPA 3.5, 대학원 수준 2
    [700, 3.5, 3], # 학생11: GRE 700, GPA 3.5, 대학원 수준 3
    [700, 3.5, 4], # 학생12: GRE 700, GPA 3.5, 대학원 수준 4    
  ])
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


array([[0.01331696],
       [0.01217468],
       [0.01118858],
       [0.01034176],
       [0.32024834],
       [0.30044815],
       [0.27891028],
       [0.25628984],
       [0.82642025],
       [0.8186683 ],
       [0.809015  ],
       [0.7972218 ]], dtype=float32)